# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, linked_variables: Dict[str, str] = {}) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negat

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables={})

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': {}}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables": {"a": [0.0, 1000.0], "b": [-1.0, 1.0]}, "constraints": {"e": ["LESS_THAN", 2.0], "f": ["GREATER_THAN", 0.0]}, "objectives": {"c": "MAXIMIZE", "d": "MINIMIZE"}, "constants": {"g": 1234}, "linked_variables": {}}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,870.212098,0.279779,1234,870.491877,869.932319,1740.983753,1739.864639
6,959.092356,0.770627,1234,959.862983,958.321729,1919.725966,1916.643457
7,82.937111,0.417800,1234,83.354911,82.519311,166.709822,165.038622
8,350.427310,0.113958,1234,350.541267,350.313352,701.082535,700.626703
9,486.916014,0.195658,1234,487.111672,486.720356,974.223344,973.440712
10,430.066283,0.283110,1234,430.349393,429.783173,860.698786,859.566345
11,915.491986,-0.852204,1234,914.639782,916.344190,1829.279563,1832.688379
12,66.325497,-0.270648,1234,66.054849,66.596146,132.109698,133.192292
13,420.983932,0.005072,1234,420.989004,420.978859,841.978008,841.957719
14,758.843341,-0.266831,1234,758.576510,759.110173,1517.153020,1518.220346


In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-870.491877,869.932319
6,-959.862983,958.321729
7,-83.354911,82.519311
8,-350.541267,350.313352
9,-487.111672,486.720356
10,-430.349393,429.783173
11,-914.639782,916.344190
12,-66.054849,66.596146
13,-420.989004,420.978859
14,-758.576510,759.110173


In [13]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-870.491877,869.932319
6,-959.862983,958.321729
7,-83.354911,82.519311
8,-350.541267,350.313352
9,-487.111672,486.720356
10,-430.349393,429.783173
11,-914.639782,916.344190
12,-66.054849,66.596146
13,-420.989004,420.978859
14,-758.576510,759.110173


In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-870.49187667,  869.93231929],
       [-959.8629829 ,  958.32172857],
       [ -83.35491112,   82.51931112],
       [-350.54126746,  350.31335156],
       [-487.1116719 ,  486.72035577],
       [-430.34939289,  429.78317254],
       [-914.63978174,  916.34418969],
       [ -66.05484902,   66.59614579],
       [-420.98900379,  420.97885938],
       [-758.57651009,  759.11017288]])

In [15]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,1738.983753,-1739.864639
6,1917.725966,-1916.643457
7,164.709822,-165.038622
8,699.082535,-700.626703
9,972.223344,-973.440712
10,858.698786,-859.566345
11,1827.279563,-1832.688379
12,130.109698,-133.192292
13,839.978008,-841.957719
14,1515.153020,-1518.220346


In [16]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
